In [12]:
from functions.functions import *
from config import model

In [13]:
def main_workflow(file_path, model, user_question):
    # Step 1: Upload Document
    docs = upload_document(file_path)
    for doc in docs:
        print(doc.page_content)
        print("-------")
    
    # Step 2: Initialize Vector Database
    vector_db = initialize_vector_db(docs)
    
    # Step 3: Retrieve Context for User's Question
    context = retrieve_context(vector_db, user_question)
    print("\n\nCONTEXT RETRIEVED:")
    for part in context:
        print(part)
        print("-------------")
    
    # Step 4: Generate Answer with Gemini API
    answer = generate_response_with_context(model, context, user_question)
    
    return answer


In [14]:
file_path = "documents/docker.txt"
user_question = "How to delete container?"

response = main_workflow(file_path, model, user_question)
print("RESPONSE:")
print(response[0])


CHUNKING OF DOCUMENT: ...
CHUNKING COMPLETE!
Instructions for Using Docker
Docker is a platform to create, deploy, and run applications inside containers. Below is a step-by-step guide:

1. Install Docker
Visit the Docker website and download Docker Desktop for your operating system.
Install Docker and follow the setup wizard.
Verify installation by running in a terminal: docker --version
-------
2. Understand Key Concepts
Image: A template for creating containers (e.g., Ubuntu, Python).
Container: A lightweight, isolated instance of an image.
Dockerfile: A script to automate image creation.
-------
3. Basic Docker Commands
Pull an image: docker pull <image_name>
Run a container: docker run -it <image_name>
List running containers: docker ps
Stop a container: docker stop <container_id>
-------
4. Create a Dockerfile
Write a Dockerfile:
Dockerfile
FROM python:3.9
WORKDIR /app
COPY . .
RUN pip install -r requirements.txt
CMD ["python", "app.py"]
Build and run: docker build -t my_app .
do

In [5]:
from functions.functions import *

search_info_to_txt("What is Monopoly game?", top=5, similarity=0.7)

Processing Root Links: 100%|██████████| 3/3 [00:16<00:00,  5.65s/it]


['https://en.wikipedia.org/wiki/History_of_Monopoly',
 'https://shop.hasbro.com/en-us/product/monopoly-game/7EABAF97-5056-9047-F577-8F4663C79E75',
 'https://www.dicebreaker.com/games/monopoly/how-to/how-to-play-monopoly']

In [3]:
get_google_links("What is Monopoly game?")

['https://en.wikipedia.org/wiki/History_of_Monopoly',
 'https://shop.hasbro.com/en-us/product/monopoly-game/7EABAF97-5056-9047-F577-8F4663C79E75',
 'https://www.dicebreaker.com/games/monopoly/how-to/how-to-play-monopoly']